In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
import glob

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Com Pandas

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
import glob

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

path = f'{diretorio_estabelecimentos}/*.csv'
all_files_estabelecimentos = glob.glob(path)

lista_df_estabelecimentos = []
for file in all_files_estabelecimentos:
    d_estabelecimento = pd.read_csv(file, 
    sep=';', 
    dtype=dtypes_ESTABELE, 
    names=ESTABELE)
    lista_df_estabelecimentos.append(d_estabelecimento)

estabelecimentos = pd.concat(lista_df_estabelecimentos, axis=0, ignore_index=True)

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

path = f'{diretorio_empresa}/*.csv'
all_files_empresa = glob.glob(path)

lista_df_empresas = []
for file in all_files_empresa:
    d_empresa = pd.read_csv(file,
        sep=';',
        dtype=dtypes_EMPRESA, 
        names=EMPRESA)
    #lista_df_empresas.append(d_empresa)

    #empresas = pd.concat(lista_df_empresas, axis=0, ignore_index=True)

    merged_data = pd.merge(estabelecimentos, d_empresa, on='CNPJ_BASE')

    chunk_size = 1000000
    for i in range(0, len(merged_data), chunk_size):
        df_chunk = merged_data.iloc[i:i+chunk_size]
        df_chunk.to_csv(f'merge_base/merged_data_{i}.csv', mode='a', index=False)

## Com Dask

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import dask.dataframe as dd
import pandas as pd

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = dd.read_csv(diretorio_estabelecimentos + '/*.csv', delimiter=';',  dtype=dtypes_ESTABELE, names=ESTABELE)
empresas = dd.read_csv(diretorio_empresa + '/*.csv', delimiter=';', dtype=dtypes_EMPRESA, names=EMPRESA)

# unindo as tabelas
merged = estabelecimentos.merge(empresas, on='CNPJ_BASE')

merged.to_csv('merge_base/merged_data.csv', single_file=True)

# mostrando as primeiras linhas
print(merged.head())

## Com Vaex

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import vaex

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = vaex.open(diretorio_estabelecimentos + '/*.csv', delimiter=';')
empresas = vaex.open(diretorio_empresa + '/*.csv', delimiter=';')

# unindo as tabelas
merged = estabelecimentos.join(empresas, on='CNPJ_BASE')

# mostrando as primeiras linhas
print(merged.head())

In [7]:
import pandas as pd
import json
dados = pd.read_csv(r'merge_base\merged_data_0.csv')
municipios = pd.read_csv('Municipios\municipios.csv', sep=';', names=['MUNICIPIO','CIDADE'])
dados = dados[['CNPJ_BASE', 'CNPJ', 'NOME_FANTASIA', 'RAZAO_SOCIAL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO', 'COMPLEMENTO', 'BAIRRO',
       'CEP', 'UF', 'MUNICIPIO']]
merged_data = pd.merge(dados, municipios, on='MUNICIPIO')
merged_data.drop(columns='MUNICIPIO', inplace=True)
merged_data.to_csv(r'merge_base\merged_data_0.csv', index=False)


In [10]:
quinhentos_dados = merged_data.iloc[0:501]
quinhentos_dados['RUA'] = quinhentos_dados['TIPO_LOGRADOURO'] + quinhentos_dados['LOGRADOURO']
quinhentos_dados.rename(columns={'UF':'ESTADO'}, inplace=True)
quinhentos_dados = quinhentos_dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA','RUA','COMPLEMENTO', 'BAIRRO','CIDADE','ESTADO','CEP']]
quinhentos_dados.to_json('amosta_cnpj.json', orient='records')

C:\Users\ABRASEL NACIONAL\AppData\Local\Temp\ipykernel_1296\2804179155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quinhentos_dados['RUA'] = quinhentos_dados['TIPO_LOGRADOURO'] + quinhentos_dados['LOGRADOURO']
C:\Users\ABRASEL NACIONAL\AppData\Local\Temp\ipykernel_1296\2804179155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quinhentos_dados.rename(columns={'UF':'ESTADO'}, inplace=True)


In [9]:
import re
file = 'base_csv_estabelecimentos\Bares e outros estabelecimentos especializados em servir bebidas com entretenimento.csv'

name_file = re.sub(".csv",'', file)
name_file = re.sub(diretorio_estabelecimentos,'', file)

name_file

error: unterminated character set at position 0